# EEG Analysis with MNE python 

the notebook covers the basic EEG/MEG pipeline for event-related analysis: loading data, epoching , averaging, and estimating cortical activity from sensor data 

In [1]:
import os 
import numpy as np 
import mne 

## Loading data 

In [2]:
sample_data_folder = mne.datasets.sample.data_path()
sample_data_raw_file = os.path.join(sample_data_folder, 'MEG', 'sample', 'sample_audvis_filt-0-40_raw.fif')

raw = mne.io.read_raw(sample_data_raw_file)

Opening raw data file /home/achintya/mne_data/MNE-sample-data/MEG/sample/sample_audvis_filt-0-40_raw.fif...
    Read a total of 4 projection items:
        PCA-v1 (1 x 102)  idle
        PCA-v2 (1 x 102)  idle
        PCA-v3 (1 x 102)  idle
        Average EEG reference (1 x 60)  idle
    Range : 6450 ... 48149 =     42.956 ...   320.665 secs
Ready.
